In [110]:
import pandas as pd
import numpy as np

In [111]:
# 보험 청구 금액을 예측 하는 인공지능 개발


In [112]:
# 1. 데이터로드
df=pd.read_csv('data/healthcare.csv', index_col= 0)

In [113]:
# 타겟 변수와 피처 분리
y = df['InsuranceClaim']
X = df.drop(columns=['InsuranceClaim'])

In [114]:
X.isna().sum()

Age          30
Gender        0
BMI          50
Region        0
Smoker        0
NumVisits    20
dtype: int64

In [115]:
y

0      70.081987
1      92.747518
2      46.794138
3      44.789132
4      71.790344
         ...    
495    79.352997
496    64.364345
497    74.631320
498    75.066459
499    45.184790
Name: InsuranceClaim, Length: 500, dtype: float64

In [116]:
X

,Age,Gender,BMI,Region,Smoker,NumVisits
0,51.0,Female,NaN,South,No,19.0
1,92.0,Female,38.074006,West,Yes,13.0
2,14.0,Male,NaN,North,No,8.0
3,NaN,Female,27.020924,West,No,7.0
4,60.0,Male,37.961368,North,No,16.0
...,...,...,...,...,...,...
495,95.0,Male,29.831966,East,No,20.0
496,47.0,Female,31.257355,West,No,13.0
497,88.0,Female,26.981973,South,Yes,11.0
498,0.0,Male,30.393551,North,Yes,20.0


In [117]:
# 2. 카테고리컬 데이터와 수치형 데이터 처리

In [118]:
# 컬럼이름을 분리해서 저장 

In [119]:
df.columns

Index(['Age', 'Gender', 'BMI', 'Region', 'Smoker', 'NumVisits',
       'InsuranceClaim'],
      dtype='object')

In [120]:
# 수치형 컬럼 이름 
numerical = ['Age','BMI','NumVisits']

In [121]:
# 카테고리컬 데이터 컬럼 이름 
categorical = ['Gender', 'Region', 'Smoker']

In [122]:
# 수치형 데이터워 NaN을, 각 컬럼의 평균값으로 채운다.
# 카테고리컬 데이터는 리에블 인코딩, 원한 잇코딩 한다

In [123]:
# 오늘은sklearn의 pipeline  라이브러리를 사용 할 것이다.
# NaN을 채우는 것은, fillna 함수 대신에, SimpleImputer를 사용한다.


In [124]:
# 수치형 컬럼의 NaN 을 각 컬럼의 평균으로 채우기 위한 라이브러리 
from sklearn.impute import SimpleImputer


imputer = SimpleImputer()

In [125]:
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

In [126]:
label_encoder = OrdinalEncoder()
oneHot_encoder = OneHotEncoder()

In [127]:
from sklearn.compose import ColumnTransformer

In [128]:
# 먼저 레이블인코딩할 컬럼과 원핫인코딩 할 컬럼을 확인 할 것이다.
df['Gender'].nunique()

2

In [129]:
df['Region'].nunique()

4

In [130]:
df['Smoker'].nunique()

2

In [131]:
label_columns = ['Gender','Smoker']
oneHot_columns = ['Region']
numeric_columns = ['Age','BMI','NumVisits']


In [132]:
preprocessor=ColumnTransformer([
    ('num', imputer, numeric_columns),
    ('label', label_encoder, label_columns),
    ('onthot', oneHot_encoder, oneHot_columns)
])

In [133]:
# 파이프라인(pipeline) 생성

In [139]:
from sklearn.ensemble import RandomForestRegressor

In [140]:
regressor = RandomForestRegressor()

In [141]:
from sklearn.pipeline import Pipeline

In [142]:
pipeline=Pipeline(steps=[('preprocessing', preprocessor),('modeling', regressor)])

In [143]:
pipeline

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num', SimpleImputer(),
                                                  ['Age', 'BMI', 'NumVisits']),
                                                 ('label', OrdinalEncoder(),
                                                  ['Gender', 'Smoker']),
                                                 ('onthot', OneHotEncoder(),
                                                  ['Region'])])),
                ('modeling', RandomForestRegressor())])

In [144]:
# 4. 학습/ 테스트 데이터 분리 후 연습

In [145]:
from sklearn.model_selection import train_test_split


In [146]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state= 5)

In [147]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num', SimpleImputer(),
                                                  ['Age', 'BMI', 'NumVisits']),
                                                 ('label', OrdinalEncoder(),
                                                  ['Gender', 'Smoker']),
                                                 ('onthot', OneHotEncoder(),
                                                  ['Region'])])),
                ('modeling', RandomForestRegressor())])

In [152]:
y_pred=pipeline.predict(X_test)

In [153]:
from sklearn.metrics import mean_squared_error, r2_score

In [154]:
mean_squared_error(y_test, y_pred)

33.560719649805485

In [155]:
r2_score( y_test, y_pred)

0.7686987309684923

In [156]:
# 파이프 라인을 이용하면 좋은점! 
##  이 파이프라인만 파일로 저장하면, 서비스 배포할때 이 파일만 있으면, 
## 예측할 데이터만 그냥 넣어주면, 파이프 라인이 내부적으로 데이터 전처리를 해서 
## 결과를 알려준다.


In [157]:
import joblib

In [158]:
joblib.dump(pipeline, 'pipeline.pkl')

['pipeline.pkl']